In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import warnings
import scipy
from scipy import signal

# Fetch data from a remote source
url = "https://raw.githubusercontent.com/ntua-el17840/Interactive-Digital-Communications/main/test-book/_static/sima.txt"
response = requests.get(url)

# Convert the response text to a numpy array of floats
s = np.array([float(line) for line in response.text.splitlines()])

# Set sampling frequency and calculate power spectral density
Fs = 8192
f, Pxx = scipy.signal.welch(s, Fs)

# Define filter characteristics
N = Fs 
H = np.concatenate((np.ones(N//8), np.zeros(N//4), np.ones(N//8)))
h = np.fft.ifft(H, n=N).real
h = np.fft.fftshift(h)

# Create filter variants of different lengths
H = np.hstack((np.ones(int(Fs/8)), np.zeros(int(Fs-Fs/4)), np.ones(int(Fs/8))))
h = np.real(np.fft.ifft(H))
middle = int(len(h)/2)
h = np.hstack((h[middle:], h[:middle]))

# Create a dictionary of filter variants
h_variants = {
    'h32': h[middle-16:middle+16],
    'h64': h[middle-32:middle+32],
    'h128': h[middle-64:middle+64],
    'h140': h[middle-70:middle+70],
    'h256': h[middle-128:middle+128],
}

# Create an output widget for displaying plots
output2 = widgets.Output()
output2.layout = Layout(width='auto', margin='0 auto')

# Function to plot the selected filter
def plot_stem(h_key):
    with output2:
        clear_output(wait=True)  # Clear the previous plots
        h_data = h_variants[h_key]
        x_values = np.arange(len(h_data))
        plt.close('all')  # Close all existing figures
        fig, ax = plt.subplots()
        markerline, stemlines, baseline = ax.stem(x_values, h_data, '-.')
        plt.setp(baseline, 'color', 'k', 'linewidth', 2)
        plt.title('Stem plot of selected filter')
        plt.xlabel('Index')
        plt.ylabel('Amplitude')
        plt.show()

# Create a dropdown widget for filter selection
dropdown = widgets.Dropdown(options=list(h_variants.keys()), value='h32', description='Filter:')

# Function to update the plot when the dropdown value changes
def update_plot(change):
    plot_stem(change['new'])

# Observe dropdown for changes
dropdown.observe(update_plot, names='value')

# Create layouts for organizing widgets
vbox_layout = Layout(display='flex', flex_flow='column', align_items='center', justify_content='space-between')
ui = widgets.HBox([dropdown], layout=Layout(align_items='center', justify_content='center'))

# Group the dropdown and output widget in a vertical box layout
vbox_final = widgets.VBox([ui, output2])

# Display the final VBox
display(vbox_final)

# Initial call to display the plot
plot_stem(dropdown.value)